In [ ]:
!pip install datasets transformers peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 19.2 MB/s eta 0:00:00


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
model_name = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map='auto',
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [ ]:
lora_config = LoraConfig(
    r = 8,
    lora_alpha = 16,
    target_modules = ['q_proj', 'v_proj'],
    lora_dropout = 0.05,
    bias = 'none',
    task_type = TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

In [ ]:
data = load_dataset('openai/gsm8k', 'main', split='train[:200]')

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [ ]:
def tokenize(batch):
  texts = [
      f'### Instruction: \n{instruction}\n### Response:\n{output}'
      for instruction, output in zip(batch['question'], batch['answer'])
      ]

  tokens = tokenizer(
      texts,
      padding = 'max_length',
      max_length = 256,
      truncation = True,
      return_tensors = 'pt'
  )

  tokens['labels'] = tokens['input_ids'].clone()

  return tokens

In [ ]:
tokenized_data = data.map(tokenize, batched=True, remove_columns=data.column_names)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
  output_dir = './tinyllama-math-lora-tutorial',
  per_device_train_batch_size = 4,
  gradient_accumulation_steps = 4,
  learning_rate = 1e-3,
  num_train_epochs = 50,
  fp16 = True,
  logging_steps = 20,
  save_strategy = 'epoch',
  report_to = 'none',
  remove_unused_columns = False,
  label_names = ['labels']
 )


In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_data,
    processing_class = tokenizer
)

In [ ]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Step,Training Loss
20,1.969800
40,0.824600
60,0.719900
80,0.634500
100,0.536900
120,0.463900
140,0.391100
160,0.306400
180,0.254700
200,0.194600


TrainOutput(global_step=650, training_loss=0.22970237832802992, metrics={'train_runtime': 1022.7762, 'train_samples_per_second': 9.777, 'train_steps_per_second': 0.636, 'total_flos': 1.590741172224e+16, 'train_loss': 0.22970237832802992, 'epoch': 50.0})

In [ ]:
model.save_pretrained('./tinyllama-lora-tuned-adapter-math')
tokenizer.save_pretrained('./tinyllama-lora-tuned-adapter-math')

('./tinyllama-lora-tuned-adapter-math/tokenizer_config.json',
 './tinyllama-lora-tuned-adapter-math/special_tokens_map.json',
 './tinyllama-lora-tuned-adapter-math/chat_template.jinja',
 './tinyllama-lora-tuned-adapter-math/tokenizer.model',
 './tinyllama-lora-tuned-adapter-math/added_tokens.json',
 './tinyllama-lora-tuned-adapter-math/tokenizer.json')

In [ ]:
import os
import math

import torch
from torch.utils.data import DataLoader

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, default_data_collator

from peft import PeftModel

In [ ]:
model_name = 'TinyLLama/TinyLlama-1.1B-Chat-v1.0'
adapter_path = './tinyllama-lora-tuned-adapter-math'

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = 'auto',
    trust_remote_code = True
).eval()

tmp_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = 'auto',
    trust_remote_code = True
)

tuned_model = PeftModel.from_pretrained(tmp_model, adapter_path)
tuned_model = tuned_model.merge_and_unload().eval()

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:

def tokenize(batch):
    texts = [
        f"### Instruction:\n{inst}\n### Response:\n{out}"
        for inst, out in zip(batch['question'], batch['answer'])
    ]

    tokens = tokenizer(
        texts,
        padding = 'max_length',
        truncation = True,
        max_length = 256,
        return_tensors = 'pt'
    )

    tokens['labels'] = tokens['input_ids'].clone()

    return tokens

In [ ]:
eval_ds = load_dataset('openai/gsm8k', 'main', split='train[:20]')
eval_ds = eval_ds.map(tokenize, batched=True, remove_columns=['question', 'answer'])
eval_ds = eval_ds.with_format('torch')

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
eval_loader = DataLoader(
    eval_ds,
    batch_size = 8,
    collate_fn = default_data_collator
)

In [ ]:
@torch.no_grad()
def compute_perplexity(model):
    losses = []

    for batch in eval_loader:
        batch = {k: v.to('cuda') for k, v in batch.items()}
        loss = model(**batch).loss
        losses.append(loss.item())

    return math.exp(sum(losses) / len(losses))

In [ ]:
print(f'Base Model Perplexity: {compute_perplexity(base_model):.2f}')
print(f'Tuned Model Perplexity: {compute_perplexity(tuned_model):.2f}')

Base Model Perplexity: 139.67
Tuned Model Perplexity: 1.13


In [ ]:
import random

raw_data = load_dataset('gsm8k', 'main', split='train[:20]')
refs = raw_data['answer']


def generate(model, instruction):
    token_ids = tokenizer(f'### Instruction:\n{instruction}\n### Response:\n', return_tensors='pt').input_ids.to('cuda')

    with torch.no_grad():
        out = model.generate(token_ids, max_new_tokens=256)

    #return tokenizer.decode(out[0], skip_special_tokens=True).split('### Response:\n')[-1].strip()
    return tokenizer.decode(out[0], skip_special_tokens=True)

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [ ]:
raw_data['question'][1]

'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?'

In [ ]:
print(generate(base_model, raw_data['question'][1]))

### Instruction:
Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
### Response:
The answer is $60.


In [ ]:
print(generate(tuned_model, raw_data['question'][1]))

### Instruction:
Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
### Response:
Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.
Working 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.
#### 10


In [ ]:
print(refs[1])

Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.
Working 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.
#### 10


In [ ]:
eval_ds = load_dataset('openai/gsm8k', 'main', split='train[200:300]')
eval_ds = eval_ds.map(tokenize, batched=True, remove_columns=['question', 'answer'])
eval_ds = eval_ds.with_format('torch')

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
eval_loader = DataLoader(
    eval_ds,
    batch_size = 8,
    collate_fn = default_data_collator
)

In [ ]:
print(f'Base Model Perplexity: {compute_perplexity(base_model):.2f}')
print(f'Tuned Model Perplexity: {compute_perplexity(tuned_model):.2f}')

Base Model Perplexity: 229.65
Tuned Model Perplexity: 8.06


In [ ]:
raw_data = load_dataset('gsm8k', 'main', split='train[200:300]')
refs = raw_data['answer']


def generate(model, instruction):
    token_ids = tokenizer(f'### Instruction:\n{instruction}\n### Response:\n', return_tensors='pt').input_ids.to('cuda')

    with torch.no_grad():
        out = model.generate(token_ids, max_new_tokens=256)

    #return tokenizer.decode(out[0], skip_special_tokens=True).split('### Response:\n')[-1].strip()
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [ ]:
raw_data['question'][0]

'Sansa is a famous artist, she can draw a portrait and sell it according to its size. She sells an 8-inch portrait for $5, and a 16-inch portrait for twice the price of the 8-inch portrait. If she sells three 8-inch portraits and five 16-inch portraits per day, how many does she earns every 3 days?'

In [ ]:
print(generate(base_model, raw_data['question'][0]))

### Instruction:
Sansa is a famous artist, she can draw a portrait and sell it according to its size. She sells an 8-inch portrait for $5, and a 16-inch portrait for twice the price of the 8-inch portrait. If she sells three 8-inch portraits and five 16-inch portraits per day, how many does she earns every 3 days?
### Response:
Sansa earns $100 per day, which means she earns $300 per week, and $1,200 per month, and $5,000 per year.


In [ ]:
print(generate(tuned_model, raw_data['question'][0]))

### Instruction:
Sansa is a famous artist, she can draw a portrait and sell it according to its size. She sells an 8-inch portrait for $5, and a 16-inch portrait for twice the price of the 8-inch portrait. If she sells three 8-inch portraits and five 16-inch portraits per day, how many does she earns every 3 days?
### Response:
She sells an 8-inch portrait for $5, so 8*5 = $<<8*5=40>>40.
She sells a 16-inch portrait for twice the price of the 8-inch portrait, or $25, so 25*2 = $<<25*2=50>>50.
There are a total of 3+5=9.
She earns a 8-inch portrait at $40 and a 16-inch portrait at $50, so she earns $<<40+50=90>>90.
Daily, she earns 90 dollars per portrait, she earns $360 per day, which is three times five times five=$<<360*5=1900>>1900.
Therefore, each day she earns $1900/3=$<<1900/3=600>>600.
#### 600


In [ ]:
print(refs[0])

Sansa earns $5 x 3 = $<<5*3=15>>15 every day by selling three 8-inch portraits.
The price of the 16-inch portrait is $5 x 2 = $<<5*2=10>>10 each.
So, she earns $10 x 5 = $<<10*5=50>>50 every day by selling five 16-inch portraits.
Her total earnings is $50 + $15 = $<<50+15=65>>65 every day.
Therefore, the total amount she earns after 3 days is $65 x 3 = $<<65*3=195>>195.
#### 195


In [ ]:
!pip install fastapi uvicorn
!pip install pyngrok

In [ ]:
%%writefile main.py
from fastapi import FastAPI
from pydantic import BaseModel
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

class Query(BaseModel):
    text: str


# --- ADD THIS SECTION ---
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allows all origins
    allow_credentials=True,
    allow_methods=["*"],  # Allows all methods
    allow_headers=["*"],  # Allows all headers
)
# ------------------------


# Load model once at startup
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
adapter_path = "./tinyllama-lora-tuned-adapter-math"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name, quantization_config=bnb_config, device_map="auto", trust_remote_code=True
)
model = PeftModel.from_pretrained(base_model, adapter_path)
model = model.merge_and_unload()
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

@app.post("/generate")
async def generate(query: Query):
    prompt = f"### Instruction:\n{query.text}\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=256, do_sample=True, temperature=0.7)

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    response = response.split("### Response:\n")[-1].strip()

    return {"response": response}

Overwriting main.py


In [ ]:
from pyngrok import ngrok
from google.colab import userdata

# Kill any process on port 8000
!fuser -k 8000/tcp

# Kill all ngrok tunnels
!pkill -f ngrok

# Get the ngrok authentication token from Colab secrets
NGROK_AUTHTOKEN = userdata.get('NGROK_AUTHTOKEN')
ngrok.set_auth_token(NGROK_AUTHTOKEN)

public_url = ngrok.connect(8000)
public_url

<NgrokTunnel: "https://7983a3acb2e2.ngrok-free.app" -> "http://localhost:8000">

In [ ]:
!uvicorn main:app --host 0.0.0.0 --port 8000

2025-11-26 13:09:43.298043: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764162583.332171   15902 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764162583.342760   15902 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764162583.365957   15902 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764162583.366002   15902 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764162583.366010   15902 computation_placer.cc:177] computation placer alr

INFO:     Shutting down
INFO:     Finished server process [15902]
ERROR:    Traceback (most recent call last):
  File "/usr/lib/python3.12/asyncio/runners.py", line 195, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/base_events.py", line 678, in run_until_complete
    self.run_forever()
  File "/usr/lib/python3.12/asyncio/base_events.py", line 645, in run_forever
    self._run_once()
  File "/usr/lib/python3.12/asyncio/base_events.py", line 1999, in _run_once
    handle._run()
  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 70, in serve
    with self.capture_signals():
         ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/contextlib

In [ ]:
# Kill any process on port 8000
!fuser -k 8000/tcp

# Kill all ngrok tunnels
!pkill -f ngrok